In [1]:
import os, sys
sys.path.append('../')
sys.path.append('../src/')

In [2]:
import numpy as np
import src.vrp_study
from src.vrp_study.data_model import Tariff, Cargo, Node
from src.vrp_study.data_model import TariffCost
from src.vrp_study.pdptw_model.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from src.vrp_study.pdptw_model.routing_model import find_optimal_paths

load c:\Users\Nikita\anaconda3\envs\crossdocking\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\Nikita\anaconda3\envs\crossdocking\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\Nikita\anaconda3\envs\crossdocking\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\Nikita\anaconda3\envs\crossdocking\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\Nikita\anaconda3\envs\crossdocking\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\Nikita\anaconda3\envs\crossdocking\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\Nikita\anaconda3\envs\crossdocking\Lib\site-packages\ortools\.libs\ortools.dll...


In [3]:
benchmark_type = 'pdp_400'
name = 'LC1_4_5.txt'

In [4]:
from typing import Optional

tariff = None
cargos: list[Cargo] = []
depo: Optional[Node] = None

In [5]:
id2info = {}
p2coordinates = {}
with open(f'../data/Li & Lim benchmark/{benchmark_type}/{name}', 'r') as file:
    for i, line in enumerate(file):
        line = line.split('\t')
        if i == 0:
            tariff = Tariff(
                id='car',
                capacity=int(line[1]),
                max_count=int(line[0]),
                cost_per_distance=[TariffCost(
                    min_dst_km=0,
                    max_dst_km=10000,
                    cost_per_km=1,
                    fixed_cost=0
                )]
            )
        else:
            c_id = int(line[0])
            x = int(line[1])
            y = int(line[2])

            mass = int(line[3])

            et = int(line[4])
            lt = int(line[5])
            st = int(line[6])

            pick_up = int(line[7])
            delivery = int(line[8])
            if pick_up == delivery:
                # print(12)
                depo = Node(
                    id=0,
                    cargo_id=c_id,
                    capacity=0,
                    service_time=0,
                    start_time=0,
                    end_time=lt,
                    coordinates=(x, y)
                )
                continue
            if pick_up == 0:
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][0] = (x, y, mass, et, lt, st, c_id, delivery)
            else:
                delivery = c_id
                c_id = pick_up
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][1] = (x, y, mass, et, lt, st, pick_up, delivery)


In [6]:
depo

Node(id=0, cargo_id=0, capacity=0, start_time=0, end_time=1501, service_time=0, coordinates=(100, 100))

In [7]:

for k, v in id2info.items():
    cargos.append(
        Cargo(
            id=k,
            nodes=[
                Node(
                    cargo_id=k,
                    id=v[i][6] if i == 0 else v[i][7],
                    capacity=v[i][2],
                    service_time=v[i][5],
                    start_time=v[i][3],
                    end_time=v[i][4],
                    coordinates=(v[i][0], v[i][1])
                )
                for i in range(2)
            ]
        )
    )

In [8]:
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [9]:

from src.vrp_study.configs import ModelConfig

routing_manager = PDRoutingManagerBuilder(
    distance_matrix = distance_matrix,
    time_matrix = time_matrix,
    model_config=ModelConfig(max_execution_time_minutes=1)
)

routing_manager.add_cargos(cargos)
routing_manager.add_tariff(tariff)

routing_manager.add_depo(depo)

routing_manager = routing_manager.build()

In [10]:
len(cargos)

211

In [11]:
from src.vrp_study.pdptw_model.solution_builder import  SolutionBuilder

sol = find_optimal_paths(routing_manager, SolutionBuilder())

2025-08-04 12:05:58.141 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 32.074999999999996)
2025-08-04 12:05:58.142 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 16.0875)
2025-08-04 12:05:58.142 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 8.09375)
2025-08-04 12:05:58.143 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 4.096875)
2025-08-04 12:05:58.144 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (1, 2.0984374999999997)
2025-08-04 12:05:58.144 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (2, 1.09921875)
2025-08-04 12:05:58.145 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (4, 0.599609375)
2025-08-04 12:05:58.145 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:118 - (4, 0.3498046875)
2025-

  0%|          | 0/1 [00:00<?, ?it/s]

2025-08-04 12:05:58.166 | INFO     | vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 9
2025-08-04 12:05:58.167 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-08-04 12:05:58.174 | INFO     | vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-08-04 12:05:58.175 | INFO     | vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-08-04 12:05:58.177 | INFO     | vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-08-04 12:05:58.178 | INFO     | vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-08-04 12:05:58.179 | INFO     | vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-08-04 12:05:58.210 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:407 - Начало решения
2025-08-04 

  0%|          | 0/1 [00:00<?, ?it/s]

2025-08-04 12:05:58.274 | INFO     | vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 13
2025-08-04 12:05:58.275 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-08-04 12:05:58.278 | INFO     | vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-08-04 12:05:58.279 | INFO     | vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-08-04 12:05:58.281 | INFO     | vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-08-04 12:05:58.282 | INFO     | vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-08-04 12:05:58.283 | INFO     | vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-08-04 12:05:58.292 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:407 - Начало решения
2025-08-04

...

2025-08-04 12:06:05.354 | INFO     | vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 3
2025-08-04 12:06:05.354 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-08-04 12:06:05.355 | INFO     | vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-08-04 12:06:05.356 | INFO     | vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-08-04 12:06:05.357 | INFO     | vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-08-04 12:06:05.358 | INFO     | vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-08-04 12:06:05.359 | INFO     | vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-08-04 12:06:05.362 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:407 - Начало решения
2025-08-04 

  0%|          | 0/1 [00:00<?, ?it/s]

2025-08-04 12:06:05.388 | INFO     | vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 3
2025-08-04 12:06:05.388 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-08-04 12:06:05.390 | INFO     | vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-08-04 12:06:05.391 | INFO     | vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-08-04 12:06:05.391 | INFO     | vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-08-04 12:06:05.392 | INFO     | vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-08-04 12:06:05.393 | INFO     | vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-08-04 12:06:05.397 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:407 - Начало решения
2025-08-04 

  0%|          | 0/1 [00:00<?, ?it/s]

2025-08-04 12:06:05.421 | INFO     | vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 3
2025-08-04 12:06:05.421 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-08-04 12:06:05.423 | INFO     | vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-08-04 12:06:05.424 | INFO     | vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-08-04 12:06:05.425 | INFO     | vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-08-04 12:06:05.425 | INFO     | vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-08-04 12:06:05.426 | INFO     | vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-08-04 12:06:05.430 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:407 - Начало решения
2025-08-04 

In [14]:
total_l = 0
for s in sol[0]:
    if len(s) > 0:
        # print(s)
        # a = routing_manager.nodes()[s[i]]
        # b = routing_manager.nodes()[s[i+1]]
        
        l = sum(routing_manager.get_distance(routing_manager.nodes()[s[i]], routing_manager.nodes()[s[i + 1]]) for i in range(len(s) - 1))
        # print(s, l)
        total_l += l
total_l

7368.258150991886

In [ ]:
for crg in cargos:
    in1 = routing_manager._node_to_inner_node[crg.nodes[0]]
    in2 = routing_manager._node_to_inner_node[crg.nodes[1]]

    start = None
    end = None
    in_time = None
    out_time = None

    for i, s in enumerate(sol[0]):
        s_time = sol[1][i]
        if len(s) == 0:
            continue

        for order, i in enumerate(s):
            if i == in1.id:
                start = i
                in_time = s_time[order]
            if i == in2.id:
                end = i
                out_time = s_time[order]

    assert start is not None and end is not None and start < end, (start, end)

    if not (crg.nodes[0].start_time <= in_time[0] and crg.nodes[0].end_time >= in_time[1]):
        print(in_time, crg.nodes[0].start_time, crg.nodes[0].end_time)
    if not (crg.nodes[1].start_time <= out_time[0] and crg.nodes[1].end_time >= out_time[1]):
        print(out_time, crg.nodes[0].start_time, crg.nodes[0].end_time)

NameError: name 'in1' is not defined